# Линейный SVM "своими руками"

## Генерируем обучающую и тестовую выборку для экспериментов

In [2]:
from sklearn.model_selection import train_test_split
from sklearn import datasets

X, y = datasets.make_classification(
    n_samples=10000, n_features=20, 
    n_classes=2, n_informative=20, 
    n_redundant=0,
    random_state=42
)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.2,
    random_state=42
)

print(len(X), len(y))
print(len(X_train))

10000 10000
8000


## Пишем свой класс для SVM

In [14]:
# coding=utf-8
import numpy as np
from sklearn.base import BaseEstimator


SVM_PARAMS_DICT = {
    'C': 10.,
    'random_state': 777,
    'iters': 10000,
    'batch_size': 100,
    'step': 0.015
}


class MySVM(BaseEstimator):
    def __init__(self, C, random_state, iters, batch_size, step):
        self.C = C
        self.random_state = random_state
        self.iters = iters
        self.batch_size = batch_size
        self.step = step

    # будем пользоваться этой функцией для подсчёта <w, x>
    def __predict(self, X):
        return np.dot(X, self.w) + self.w0

    # sklearn нужно, чтобы predict возвращал классы, поэтому оборачиваем наш __predict в это
    def predict(self, X):
        res = self.__predict(X)
        res[res > 0] = 1
        res[res < 0] = 0
        return res
    
    # производная регуляризатора
    def der_reg(self):
        return 1. / self.C * self.w

    # будем считать стохастический градиент не на одном элементе, а сразу на пачке (чтобы было эффективнее)
    def der_loss(self, x, y):
        # s.shape == (batch_size, features)
        # y.shape == (batch_size,)

        # считаем производную по каждой координате на каждом объекте
        pred = self.predict(x)

        # занулить производные там, где отступ > 1
        pred[pred > 1] = 0

        # для масштаба возвращаем средний градиент по пачке
        
        return np.mean(pred)

    def fit(self, X_train, y_train):
        # RandomState для воспроизводитмости
        random_gen = np.random.RandomState(self.random_state)
        
        # получаем размерности матрицы
        size, dim = X_train.shape
        
        # случайная начальная инициализация
        self.w = random_gen.rand(dim)
        self.w0 = random_gen.randn()

        for _ in range(self.iters):  
            # берём случайный набор элементов
            rand_indices = random_gen.choice(size, self.batch_size)
            x = X_train[rand_indices]
            y = y_train[rand_indices] * 2 - 1 # исходные метки классов это 0/1 а нам надо -1/1
            
#             print(x.shape, y.shape)
            # считаем производные
            self.w -= self.step * np.dot(x.T, y) * self.der_loss(x,y)

            # обновляемся по антиградиенту
            self.w0 -= self.step * y * self.der_loss(x,y)

        # метод fit для sklearn должен возвращать self
        return self

## Пробуем обучить наш классификатор и посмотреть на качество на тесте

In [13]:
model = MySVM(10, 777, 10000, 100, 0.015)
model.fit(X_train, y_train)
print(model.w, model.w0)

[   67.1418887   3800.64446516   -66.38194882 -4142.06675674   105.73930232
 -3633.14305382 -3547.69120486    50.57275398  3544.54876303  3591.66311292
   432.74519899 -3545.85226361 -3571.9973232  -3785.44152787  7258.71531523
  -311.2737768  -3398.01172686 -3509.10902984   254.49040724 -3647.94009493] [-0.32856751 -0.73506751 -0.66846751 -0.55716751 -0.44346751 -0.29856751
 -0.88296751  0.20453249 -0.40746751 -0.38196751 -1.33596751 -1.38306751
 -0.35496751  0.23423249 -0.32946751 -0.64056751 -1.21086751 -1.54956751
 -0.59616751  0.16043249  0.14633249  1.69853249 -0.17286751 -0.54726751
 -1.09086751 -1.04616751 -0.97116751  0.21323249 -1.08546751 -0.57576751
 -0.59526751 -0.74346751  0.91133249 -0.71496751 -0.51816751 -0.55776751
 -0.99816751  0.17393249  0.21983249  1.00523249  0.30473249 -0.92166751
  1.41743249  0.27563249 -0.72396751  0.49883249  0.84863249 -1.13556751
 -0.19206751  0.40283249  0.16103249  0.41933249 -1.71396751 -0.07386751
  0.62633249  1.09883249  0.81593249 -

In [15]:
predictions = model.predict(X_test)

ValueError: operands could not be broadcast together with shapes (2000,) (100,) 

In [11]:
print(predictions)

[0 1 0 ..., 1 1 1]


In [12]:
print(y_test, len(y_test), sum(y_test))

[1 0 1 ..., 1 0 1] 2000 991


In [13]:
print(len(predictions), sum(predictions))

2000 1186


In [14]:
print(sum(predictions == y_test) / float(len(y_test)))

0.6085
